## Deliverable 2. Create a Customer Travel Destinations Map.

In [3]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [4]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Database/weather_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Punta Arenas,CL,-53.1500,-70.9167,35.71,87,100,6.91,overcast clouds
1,1,Port Alfred,ZA,-33.5906,26.8910,60.62,83,20,6.60,few clouds
2,2,Paracatu,BR,-17.2222,-46.8747,77.40,32,70,4.72,broken clouds
3,3,Jamestown,US,42.0970,-79.2353,82.02,65,0,9.22,clear sky
4,4,Coquimbo,CL,-29.9533,-71.3436,55.13,76,0,6.91,clear sky


In [5]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))


What is the minimum temperature you would like for your trip? 70
What is the maximum temperature you would like for your trip? 90


In [6]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
2,2,Paracatu,BR,-17.2222,-46.8747,77.40,32,70,4.72,broken clouds
3,3,Jamestown,US,42.0970,-79.2353,82.02,65,0,9.22,clear sky
5,5,Souillac,MU,-20.5167,57.5167,70.18,83,40,6.91,light rain
9,9,Bathsheba,BB,13.2167,-59.5167,81.21,83,75,19.57,broken clouds
10,10,Kavaratti,IN,10.5669,72.6420,80.62,83,100,23.29,overcast clouds
14,14,Port Blair,IN,11.6667,92.7500,81.12,82,100,19.33,overcast clouds
15,15,Atuona,PF,-9.8000,-139.0333,77.65,73,30,15.03,scattered clouds
23,23,Bambanglipuro,ID,-7.9500,110.2833,76.50,82,43,4.43,scattered clouds
24,24,Chau Doc,VN,10.7000,105.1167,75.74,85,100,5.82,overcast clouds
25,25,Rikitea,PF,-23.1203,-134.9692,73.04,79,100,20.80,light rain


In [7]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                344
City                   344
Country                344
Lat                    344
Lng                    344
Max Temp               344
Humidity               344
Cloudiness             344
Wind Speed             344
Current Description    344
dtype: int64

In [8]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df
clean_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
2,2,Paracatu,BR,-17.2222,-46.8747,77.40,32,70,4.72,broken clouds
3,3,Jamestown,US,42.0970,-79.2353,82.02,65,0,9.22,clear sky
5,5,Souillac,MU,-20.5167,57.5167,70.18,83,40,6.91,light rain
9,9,Bathsheba,BB,13.2167,-59.5167,81.21,83,75,19.57,broken clouds
10,10,Kavaratti,IN,10.5669,72.6420,80.62,83,100,23.29,overcast clouds


In [9]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Paracatu,BR,77.40,broken clouds,-17.2222,-46.8747,
3,Jamestown,US,82.02,clear sky,42.0970,-79.2353,
5,Souillac,MU,70.18,light rain,-20.5167,57.5167,
9,Bathsheba,BB,81.21,broken clouds,13.2167,-59.5167,
10,Kavaratti,IN,80.62,overcast clouds,10.5669,72.6420,
14,Port Blair,IN,81.12,overcast clouds,11.6667,92.7500,
15,Atuona,PF,77.65,scattered clouds,-9.8000,-139.0333,
23,Bambanglipuro,ID,76.50,scattered clouds,-7.9500,110.2833,
24,Chau Doc,VN,75.74,overcast clouds,10.7000,105.1167,
25,Rikitea,PF,73.04,light rain,-23.1203,-134.9692,


In [10]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.

for index, row in hotel_df.iterrows():

    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
     # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [11]:
hotel_df.tail(30)


,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
643,Aloleng,PH,77.88,moderate rain,16.1307,119.7824,Steve Walden and Emma Sison-Walden Residence
644,Batticaloa,LK,79.25,overcast clouds,7.7102,81.6924,Grand Star Guest House
645,The Valley,AI,84.06,clear sky,18.2170,-63.0578,CeBlue Villas & Beach Resort
646,Papara,BR,78.69,clear sky,-3.9839,-38.7273,CAMPOS MARANGUAPE
649,Samana,DO,80.31,overcast clouds,19.2056,-69.3369,The Bannister - Hotel & Yacht Club
651,Porto Torres,IT,73.58,few clouds,40.8350,8.3972,Hotel Elisa
652,Dafeng,CN,81.45,few clouds,23.4517,108.5881,Shenglong Hotel
656,Ranong,TH,78.73,overcast clouds,9.9658,98.6348,Numsai Khaosuay Resort Ranong
657,Fort Saint James,CA,77.04,clear sky,54.4333,-124.2500,The View Hotel
658,Shahrud,IR,71.91,clear sky,36.4182,54.9763,Royal Hotel


In [12]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df.loc[(hotel_df['Hotel Name'] == '')] = np.NaN
clean_hotel_df = hotel_df.dropna(subset=['Hotel Name'])
clean_hotel_df.count()

City                   321
Country                321
Max Temp               321
Current Description    321
Lat                    321
Lng                    321
Hotel Name             321
dtype: int64

In [13]:
# 8a. Create the output File (CSV)
output_data_file = "vacation_search/WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [14]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [15]:
# 11a. Add a marker layer for each city to the map. 

fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(markers)

# 11b. Display the figure
fig


Figure(layout=FigureLayout(height='420px'))